<a href="https://colab.research.google.com/github/zhuzihan728/metal-binding-site-prediction/blob/main/cluster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libs

In [ ]:
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 30.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from Bio import SeqIO

# Extract datasets

In [ ]:
!tar -xvf /content/drive/MyDrive/FYP/miniconda -C /root

流式输出内容被截断，只能显示最后 5000 行内容。
miniconda/lib/python3.7/site-packages/conda_env/cli/__pycache__/main_config.cpython-37.pyc
miniconda/lib/python3.7/site-packages/conda_env/cli/main_config.py
miniconda/lib/python3.7/site-packages/conda_env/cli/main.py
miniconda/lib/python3.7/site-packages/conda_env/cli/main_vars.py
miniconda/lib/python3.7/site-packages/conda_env/exceptions.py
miniconda/lib/python3.7/site-packages/conda_env/installers/
miniconda/lib/python3.7/site-packages/conda_env/installers/__init__.py
miniconda/lib/python3.7/site-packages/conda_env/installers/conda.py
miniconda/lib/python3.7/site-packages/conda_env/installers/base.py
miniconda/lib/python3.7/site-packages/conda_env/installers/__pycache__/
miniconda/lib/python3.7/site-packages/conda_env/installers/__pycache__/pip.cpython-37.pyc
miniconda/lib/python3.7/site-packages/conda_env/installers/__pycache__/conda.cpython-37.pyc
miniconda/lib/python3.7/site-packages/conda_env/installers/__pycache__/__init__.cpython-37.pyc
miniconda/lib

In [ ]:
!tar -xvf /content/drive/MyDrive/FYP/uniprot_datasets -C /content

ChEBI-IDs_for_metal_binding.tsv
NEG_clustered_rep_seq.fasta
NEG_TRAIN.fasta
POS_TRAIN.fasta
POS_TRAIN_FULL.fasta
POS_TRAIN_FULL.tsv
POS_TRAIN.tsv
filtered_combined.fasta
trimed_combined.fasta


In [ ]:
total_len = len(list(SeqIO.parse("/content/trimed_combined.fasta", "fasta")))
print("Full data set size: ", total_len)

Full data set size:  177367


# Helper functions

In [ ]:
def check_metal(seqs, metal, anno, metal_count_df):
  cnt = 0
  temp = anno.loc[anno['Accession'].isin(seqs)]
  temp1 = temp['ChEBI-ID'].value_counts().to_frame().reset_index()
  row = temp1[temp1['index'] == metal]['ChEBI-ID']
  cnt = 0 if len(row) == 0 else int(row)
  per = cnt / int(metal_count_df[metal_count_df['ChEBI-ID'] == metal]['count'])
  return per

In [ ]:
def check_metal_num(seqs, metal, anno):
  cnt = 0
  temp = anno.loc[anno['Accession'].isin(seqs)]
  temp1 = temp['ChEBI-ID'].value_counts().to_frame().reset_index()
  row = temp1[temp1['index'] == metal]['ChEBI-ID']
  cnt = 0 if len(row) == 0 else int(row)
  return cnt

In [ ]:
def check_metal_specific_residue_proportion(acc_ls, source = 'POS_TRAIN_FULL.tsv', use_trimed=True):
  anno = pd.read_csv(source, sep='\t')
  metal_count_df = anno['ChEBI-ID'].value_counts().to_frame().reset_index()
  metal_count_df.columns = ['ChEBI-ID', 'count']
  if use_trimed:
    acc, _ = fasta2acc_seq_ls("trimed_combined.fasta")
    temp_cnt = []
    for i in metal_count_df['ChEBI-ID']:
      temp_cnt.append(check_metal_num(acc, i, anno))
    metal_count_df = pd.DataFrame({'ChEBI-ID': metal_count_df['ChEBI-ID'], 'count': temp_cnt})
  metal_id_name_df = pd.read_csv('ChEBI-IDs_for_metal_binding.tsv', sep='\t')
  for metal in metal_count_df['ChEBI-ID'].unique():
    metal_name = metal_id_name_df[metal_id_name_df['ChEBI-ID']==metal]['Name'].iloc[0]
    per = check_metal(acc_ls, metal, anno, metal_count_df) 
    num = int(metal_count_df[metal_count_df['ChEBI-ID'] == metal]['count'])
    print(f'{metal:12}| {metal_name:29} | num: {int(num*per):6} | %: {per}')

In [ ]:
def write_seq_ls2fasta(file_out, ls, source):
  with open(file_out, 'w') as f_out:
    for seq_record in SeqIO.parse(source, "fasta"):
      seq_acc = seq_record.id.split('|')[1]
      if seq_acc in ls:
        r = SeqIO.write(seq_record, f_out, 'fasta')

        if r!=1: 
          print('Error while writing sequence: ' + seq_acc)
        else:
          print(f'writing {seq_acc} to train fasta file.')

In [ ]:
def fasta2acc_seq_ls(path):
  acc = []
  seq = []

  for seq_record in SeqIO.parse(path, "fasta"):
    acc.append(seq_record.id.split('|')[1])
    seq.append(str(seq_record.seq))
  return acc, seq

In [ ]:
def check_pos_neg_proportion(ls):
  total_num = len(ls)
  
  acc, _ = fasta2acc_seq_ls("POS_TRAIN_FULL.fasta")
  inter = set(acc).intersection(ls)
  pos_num = len(inter)
  neg_num = total_num - pos_num
  pos_portion = pos_num/total_num
  neg_portion = neg_num/total_num
  print(f'total seq in the set: {total_num}')
  print(f'proportion over full dataset: {total_num/total_len}')
  print(f'pos: {pos_num} %: {pos_portion}')
  print(f'neg: {neg_num} %: {neg_portion}')
  return total_num, pos_num, neg_num, pos_portion, neg_portion

In [ ]:
def identity_above_threshold(m8file, thres):
  data = pd.read_csv(m8file, sep="\t", index_col=False, header=None)
  data.columns = ["query", "target","sequence identity","alignment length","mismatch","gap opening", "query domain start position", "end position","target domain start position", "end position", "evalue", "bit score"]
  
  seq_above_thres = data[data["sequence identity"] > thres]["query"].unique()
  seq_below_thres = data[~data["query"].isin(seq_above_thres)]["query"].unique()
  # print(data[data["sequence identity"] > thres]["sequence identity"].unique())
  all_seq = data["query"].unique()
  proportion = len(seq_above_thres) / len(all_seq)
  print(len(all_seq) == len(seq_above_thres) + len(seq_below_thres))
  return seq_above_thres, seq_below_thres, proportion

In [ ]:
def read_fasta(fasta_path, split_char="|", id_field=1):
    '''
        Reads in fasta file containing multiple sequences.
        Split_char and id_field allow to control identifier extraction from header.
        E.g.: set split_char="|" and id_field=1 for SwissProt/UniProt Headers.
        Returns dictionary holding multiple sequences or only single 
        sequence, depending on input file.
    '''
    
    seqs = dict()
    with open( fasta_path, 'r' ) as fasta_f:
        for line in fasta_f:
            # get uniprot ID from header and create new entry
            if line.startswith('>'):
                uniprot_id = line.replace('>', '').strip().split(split_char)[id_field]
                # replace tokens that are mis-interpreted when loading h5
                uniprot_id = uniprot_id.replace("/","_").replace(".","_")
                seqs[ uniprot_id ] = ''
            else:
                # repl. all whie-space chars and join seqs spanning multiple lines, drop gaps and cast to upper-case
                seq= ''.join( line.split() ).upper().replace("-","")
                # repl. all non-standard AAs and map them to unknown/X
                seq = seq.replace('U','X').replace('Z','X').replace('O','X')
                seqs[ uniprot_id ] += seq 
    example_id=next(iter(seqs))
    print("Read {} sequences.".format(len(seqs)))
    print("Example:\n{}\n{}".format(example_id,seqs[example_id]))

    return seqs

In [ ]:
def dataset_metal_binding_summary(acc_ls, source = 'POS_TRAIN_FULL.tsv'):
  total_num = len(acc_ls)
  print(f'total seq in the set: {total_num}')

  all_pos_acc_ls, _ = fasta2acc_seq_ls("POS_TRAIN_FULL.fasta")
  metals = {'CHEBI:29105':0,'CHEBI:18420':1,'CHEBI:49883':2,'CHEBI:29108':3,'CHEBI:29035':4,'CHEBI:60240':5,'CHEBI:24875':6,'CHEBI:190135':7,'CHEBI:23378':8,'CHEBI:29103':9,'CHEBI:49786':10,'CHEBI:29101':11,'CHEBI:29034':12,'CHEBI:30408':13,'CHEBI:29036':14,'CHEBI:29033':15,'CHEBI:21137':16,'CHEBI:49552':17,'CHEBI:48775':18,'CHEBI:48828':19,'CHEBI:21143':20,'CHEBI:25213':21,'CHEBI:47739':22,'CHEBI:16793':23,'CHEBI:177874':24,'CHEBI:60400':25,'CHEBI:49415':26,'CHEBI:60504':27,'CHEBI:49713':28}
  anno = pd.read_csv(source, sep='\t')
  metal_count_df = anno['ChEBI-ID'].value_counts().to_frame().reset_index()
  metal_count_df.columns = ['ChEBI-ID', 'count']
  metal_id_name_df = pd.read_csv('ChEBI-IDs_for_metal_binding.tsv', sep='\t')
  prot_counter = [0]*29 
  res_counter = [0]*29
  pos_acc = set(all_pos_acc_ls).intersection(acc_ls)
  for i, metal in enumerate(metals):
    metal_name = metal_id_name_df[metal_id_name_df['ChEBI-ID']==metal]['Name'].iloc[0]
    temp = anno[anno['ChEBI-ID'] == metal]
    prot_counter[i] += len(temp[temp['Accession'].isin(pos_acc)]['Accession'].unique())
    res_counter[i] += check_metal_num(acc_ls, metal, anno)
    total_res_num = int(metal_count_df[metal_count_df['ChEBI-ID'] == metal]['count'])
    print(f"{metal:13}|{metal_name:30}|#p: {prot_counter[i]:10}|#residue: {res_counter[i]:6}|%residue/all: {res_counter[i]/total_res_num:{5}.{3}}")
  print(f"#non-binding protein: {total_num-len(pos_acc)}")
  return prot_counter, res_counter


# Clustering by MMSEQS

In [ ]:
%alias activate $HOME/miniconda/bin/activate

In [ ]:
%alias mmseqs $HOME/miniconda/pkgs/mmseqs2-14.7e284-pl5321hf1761c0_0/bin/mmseqs

In [ ]:
activate tutorial

In [ ]:
mmseqs

MMseqs2 (Many against Many sequence searching) is an open-source software suite for very fast, 
parallelized protein sequence searches and clustering of huge protein sequence data sets.

Please cite: M. Steinegger and J. Soding. MMseqs2 enables sensitive protein sequence searching for the analysis of massive data sets. Nature Biotechnology, doi:10.1038/nbt.3988 (2017).

MMseqs2 Version: 14.7e284
© Martin Steinegger (martin.steinegger@snu.ac.kr)

usage: mmseqs <command> [<args>]

Easy workflows for plain text input/output
  easy-search       	Sensitive homology search
  easy-cluster      	Slower, sensitive clustering
  easy-linclust     	Fast linear time cluster, less sensitive clustering
  easy-taxonomy     	Taxonomic classification
  easy-rbh          	Find reciprocal best hit

Main workflows for database input/output
  search            	Sensitive homology search
  map               	Map nearly identical sequences
  rbh               	Reciprocal best hit search
  linclust          	F

In [ ]:
mmseqs easy-cluster

usage: mmseqs easy-cluster <i:fastaFile1[.gz|.bz2]> ... <i:fastaFileN[.gz|.bz2]> <o:clusterPrefix> <tmpDir> [options]
options:                               
 -c FLOAT                       List matches above this fraction of aligned (covered) residues (see --cov-mode) [0.800]
 --cov-mode INT                 0: coverage of query and target
                                1: coverage of target
                                2: coverage of query
                                3: target seq. length has to be at least x% of query length
                                4: query seq. length has to be at least x% of target length
                                5: short seq. needs to be at least x% of the other seq. length [0]
 --alignment-mode INT           How to compute the alignment:
                                0: automatic
                                1: only score and end_pos
                                2: also start_pos and cov
                                3: also seq.i

In [ ]:
mmseqs easy-cluster trimed_combined.fasta assembly_clustered tmp --cov-mode 5 -c 0.25 --min-seq-id 0.4

Create directory tmp
easy-cluster trimed_combined.fasta assembly_clustered tmp --cov-mode 5 -c 0.25 --min-seq-id 0.4 

MMseqs Version:                     	14.7e284
Substitution matrix                 	aa:blosum62.out,nucl:nucleotide.out
Seed substitution matrix            	aa:VTML80.out,nucl:nucleotide.out
Sensitivity                         	4
k-mer length                        	0
k-score                             	seq:2147483647,prof:2147483647
Alphabet size                       	aa:21,nucl:5
Max sequence length                 	65535
Max results per query               	20
Split database                      	0
Split mode                          	2
Split memory limit                  	0
Coverage threshold                  	0.25
Coverage mode                       	5
Compositional bias                  	1
Compositional bias                  	1
Diagonal scoring                    	true
Exact k-mer matching                	0
Mask residues                       	1
Mask residues pr

# Reading data

## reading dataset and annotations

acc: protein accessions in trimed_combined.fasta

In [ ]:
acc, _ = fasta2acc_seq_ls("trimed_combined.fasta")

anno: the annotation dataframe POS_TRAIN_FULL.tsv

In [ ]:
anno = pd.read_csv('POS_TRAIN_FULL.tsv', sep='\t')
anno

,Accession,Evidence,ChEBI-ID,Position
0,P0C6X8,ECO:0000255,CHEBI:29105,6190
1,P0C6X8,ECO:0000255,CHEBI:29105,6376
2,Q9RS27,ECO:0000255,CHEBI:29105,693
3,Q5HLY0,ECO:0000255,CHEBI:18420,97
4,Q9I2T1,ECO:0000255,CHEBI:29105,170
...,...,...,...,...
409984,C5JYZ5,ECO:0000255,CHEBI:60240,318
409985,C5JYZ5,ECO:0000255,CHEBI:60240,446
409986,A7FLX7,ECO:0000255,CHEBI:60240,8
409987,A7FLX7,ECO:0000255,CHEBI:60240,44


metal_count_df: metal and the number of residues binding that metal from POS_TRAIN_FULL

In [ ]:
metal_count_df = anno['ChEBI-ID'].value_counts().to_frame().reset_index()
metal_count_df.columns = ['ChEBI-ID', 'count']
metal_count_df

,ChEBI-ID,count
0,CHEBI:29105,133807
1,CHEBI:18420,89414
2,CHEBI:49883,51607
3,CHEBI:29108,42179
4,CHEBI:29035,22438
5,CHEBI:60240,18192
6,CHEBI:24875,17448
7,CHEBI:190135,9036
8,CHEBI:23378,7395
9,CHEBI:29103,6266


In [ ]:
temp_cnt = []
for i in metal_count_df['ChEBI-ID']:
  temp_cnt.append(check_metal_num(acc, i, anno))
metal_count_df = pd.DataFrame({'ChEBI-ID': metal_count_df['ChEBI-ID'], 'count': temp_cnt})
metal_count_df

,ChEBI-ID,count
0,CHEBI:29105,109557
1,CHEBI:18420,80898
2,CHEBI:49883,49857
3,CHEBI:29108,37010
4,CHEBI:29035,21883
5,CHEBI:60240,17704
6,CHEBI:24875,16869
7,CHEBI:190135,8595
8,CHEBI:23378,7001
9,CHEBI:29103,6218


## retrive cluster results

In [ ]:
print('number of prot seqs:', len(list(SeqIO.parse("trimed_combined.fasta", "fasta"))))
# 262004

number of prot seqs: 177367


In [ ]:
print('number of clusters:', len(list(SeqIO.parse("assembly_clustered_rep_seq.fasta", "fasta"))))
# 38717

number of clusters: 33323


clusters: dataframe of [rep accession, a protein accession in the rep's cluster]


In [ ]:
clusters = pd.read_csv('assembly_clustered_cluster.tsv', sep='\t', header=None)
clusters.columns = ['Rep', 'Accession']
clusters

,Rep,Accession
0,Q4JAP7,Q4JAP7
1,Q4JAP7,Q976K1
2,Q4JAP7,C3MJ44
3,Q4JAP7,C3MYT6
4,Q4JAP7,C3MZU1
...,...,...
177362,Q54316,P71404
177363,Q54316,P33416
177364,Q54316,O78410
177365,Q54316,Q2FV74


In [ ]:
print('number of reps: %d' % len(clusters['Rep'].unique())) # check if #rep == #cluster

number of reps: 33323


In [ ]:
print('number of clustered seqs: %d' % len(clusters)) # check if #clustered seqs == #seqs

number of clustered seqs: 177367


## Extract metal binding information of the clusters

cluster_label: merging cluster with annotations, **drop clusters where there is no metal-binding sequence at all**. \
dataframe of [rep acc, protein acc, evidence of the protein, metal binding to the protein, position of binding site]

In [ ]:
cluster_label = pd.merge(clusters, anno, on = 'Accession', how = "outer")
cluster_label.dropna(axis=0, how='any', inplace=True)
cluster_label

,Rep,Accession,Evidence,ChEBI-ID,Position
0,Q4JAP7,Q4JAP7,ECO:0000255,CHEBI:29105,317.0
1,Q4JAP7,Q4JAP7,ECO:0000255,CHEBI:29105,67.0
2,Q4JAP7,Q4JAP7,ECO:0000255,CHEBI:29105,145.0
3,Q4JAP7,Q4JAP7,ECO:0000255,CHEBI:29105,122.0
4,Q4JAP7,Q4JAP7,ECO:0000255,CHEBI:29105,91.0
...,...,...,...,...,...
457223,B5E972,Q7NML6,ECO:0000255,CHEBI:49883,50.0
457296,Q08DM7,A0A3Q0KDV9,ECO:0000269,CHEBI:18420,100.0
457297,Q08DM7,A0A3Q0KDV9,ECO:0007744,CHEBI:18420,55.0
457298,Q08DM7,A0A3Q0KDV9,ECO:0000269,CHEBI:18420,55.0


## Record the number of binding residues with different metals for clusters and sequences

For each cluster (represented by the rep prot accession) where there are metal-binding sequences, count the number of binding residues with each metal. \

rep_metal: dataframe of [cluster rep, metal type, number of binding residues with the metal in the cluster] \

Used as a record table for future data splitting.

In [ ]:
rep_metal = cluster_label[['Rep', 'ChEBI-ID']].value_counts().to_frame().reset_index()
rep_metal.columns = ['Rep', 'ChEBI-ID', 'count']
rep_metal

,Rep,ChEBI-ID,count
0,A1KR91,CHEBI:29105,4395
1,B9MQ24,CHEBI:49883,4150
2,P53441,CHEBI:29108,3312
3,A5FRM5,CHEBI:18420,2693
4,P47944,CHEBI:60240,2674
...,...,...,...
6460,Q7UU71,CHEBI:18420,1
6461,Q9YD10,CHEBI:18420,1
6462,A9NGI4,CHEBI:29103,1
6463,P45516,CHEBI:18420,1


rep_Allmetal: a temporary dataframe of [rep of a cluster, number of binding residues in the cluster regardless of metal type]

reps: a list containing the accessions of all reps whose cluster contains metal-binding sequences, shuffle it for future data splitting.

In [ ]:
rep_Allmetal = cluster_label['Rep'].value_counts().to_frame().reset_index()
rep_Allmetal.columns = ['Rep', 'count']
reps = list(rep_Allmetal['Rep'].unique())
import random
random.seed(42)
random.shuffle(reps)
reps[:10]

['P0C9E0',
 'Q09833',
 'Q58028',
 'Q6Z1I5',
 'Q89ZI8',
 'Q7MXR4',
 'P19515',
 'Q8F874',
 'B2V955',
 'Q9D6H2']

For each metal-binding sequence, count the number of binding residues with each metal. \

seq_metal: dataframe of [prot sequence, metal type, number of binding residues with the metal] \

Used as a record table for future data splitting.

In [ ]:
seq_metal = cluster_label[['Accession', 'ChEBI-ID']].value_counts().to_frame().reset_index()
seq_metal.columns = ['Accession', 'ChEBI-ID', 'count']
seq_metal

,Accession,ChEBI-ID,count
0,O31526,CHEBI:29108,88
1,E0VIU9,CHEBI:29105,84
2,O31527,CHEBI:29108,81
3,Q9Y4X5,CHEBI:29105,72
4,Q7KTX7,CHEBI:29105,63
...,...,...,...
94453,Q8ZKU9,CHEBI:18420,1
94454,B6YRP0,CHEBI:18420,1
94455,C4LIT2,CHEBI:18420,1
94456,A2SIT7,CHEBI:18420,1


seq_Allmetal: a temporary dataframe of [a prot seq, number of binding residues to the seq regardless of metal type]

seqs: a list containing the accessions of all metal-binding protein seqs in POS_TRAIN_FULL.fasta, shuffle it for future data splitting.

In [ ]:
seq_Allmetal = cluster_label['Accession'].value_counts().to_frame().reset_index()
seq_Allmetal.columns = ['Accession', 'count']
seqs = list(seq_Allmetal['Accession'].unique())
random.shuffle(seqs)
seqs[:10]

['Q57290',
 'Q7N8P3',
 'A4QDS8',
 'A1TSE0',
 'P95646',
 'Q6NTW5',
 'B7GU04',
 'Q5HVN3',
 'B5BCN2',
 'C0HJX1']

In [ ]:
print('number of clusters containing metalloprotein: %d' % len(reps))

number of clusters containing metalloprotein: 5896


non_metal_reps: all reps whose cluster contains no metal-binding sequences at all.

In [ ]:
non_metal_reps = []
for i in clusters['Rep'].unique():
  if i not in reps:
    non_metal_reps.append(i)
print('number of clusters not containing metalloprotein: %d' % len(non_metal_reps))

number of clusters not containing metalloprotein: 27427


# Data splitting

- Goal: split a test dataset from the whole dataset with a test:trainval ratio 1:9, making sure that in the test set, the number of metal-binding residues for each metal type is roughly 1:9 of the trainval set, this also ensures that metal-binding residues for each metal type appears at least once in the test set. \


- procedure
  1. establish a table (metal_count_df) from the annotation POS_TRAIN_FULL.fasta. The table records every metal and the number of residues binding to the metal.
  2. establish a table (metal_count_test) from the annotation POS_TRAIN_FULL.fasta. The table records every metal and the number of residues binding to the metal * 0.1, rounded.
  3. iterate random shuffled clusters containing at least one metal-binding protein, for each cluster, decide if it should go to trainval or test set by: \
    a. for each metal type
      - retrive the number of residues binding to it in that cluster. 
      - compare the number to the record in metal_count_test, if the number is greater than the record (+10 flexibility), append to trainval set.
      - compare the number to the record in metal_count_df, if the number equals the record, mearning all residues binding to the metal belong to this cluster, append to trainval set. 

    b. If the cluster is not appended to trainvalset in a., append to test set, update metal_count_test by subtracting the numbers of newly added residues of each metal type from the corresponding records.
  4. For clusters containing no metal-binding protein at all, randomly assign them to trainval or test set with probs 0.9, 0.1.
  5. replace clusters in the trainval and test set by proteins in the clusters.
  6. For each metal, check the #binding residues in test set/#binding residues in full set, if the proportion is smaller than 0.99, randomly grab to test set a protein from trainval set that binds the metal, this is done iteratively until the proportion hits 0.99.

## cluster splitting

In [ ]:
import numpy as np

In [ ]:
import math

metal_count_test = metal_count_df.copy()
metal_count_test['count'] = metal_count_test['count']*0.2//1+1

metal_test = set()
test = []
trainval = []
for i in reps:
  print(f'----- decide for cluster {i} -----')

  temp = rep_metal[rep_metal['Rep'] == i].reset_index()
  print(f'cluster {i} has annotations:\n{temp}')
  flag = 1
  for m in temp['ChEBI-ID']:
    num = temp[temp['ChEBI-ID']==m].iloc[0,3]
    cur = metal_count_test.loc[metal_count_test['ChEBI-ID']==m].iloc[0,1]
    max_num = metal_count_df.loc[metal_count_df['ChEBI-ID']==m].iloc[0,1]
    if cur + 10 < num or num == max_num:
      print(f'metal {m} exceed maximum, cluster {i} -> trainval')
      trainval.append(i)
      flag = 0
      break
  if flag:
    test.append(i)
    print(f'cluster {i} -> test')
    for n in temp['ChEBI-ID']: 
        print(f"the cluster has metal {n}, number {temp[temp['ChEBI-ID']==n].iloc[0,3]}")
        print(metal_count_test)
        metal_count_test.loc[metal_count_test['ChEBI-ID']==n, 'count'] -= temp[temp['ChEBI-ID']==n].iloc[0,3]
        print(metal_count_test)
        metal_test.add(n)
        print(f"update metal presented in test set: {metal_test}")
  print('\n\n')

randnums = np.random.randint(0, len(non_metal_reps), int(0.1*len(non_metal_reps)))
for i, v in enumerate(non_metal_reps):
  if i in randnums:
    test.append(v)
  else:
    trainval.append(v)
print(test)
print(trainval)



流式输出内容被截断，只能显示最后 5000 行内容。
metal CHEBI:29108 exceed maximum, cluster P54670 -> trainval



----- decide for cluster Q6A8U1 -----
cluster Q6A8U1 has annotations:
   index     Rep     ChEBI-ID  count
0   3399  Q6A8U1  CHEBI:18420      8
metal CHEBI:18420 exceed maximum, cluster Q6A8U1 -> trainval



----- decide for cluster O83153 -----
cluster O83153 has annotations:
   index     Rep     ChEBI-ID  count
0   4315  O83153  CHEBI:18420      5
metal CHEBI:18420 exceed maximum, cluster O83153 -> trainval



----- decide for cluster O67883 -----
cluster O67883 has annotations:
   index     Rep     ChEBI-ID  count
0   3312  O67883  CHEBI:18420      8
metal CHEBI:18420 exceed maximum, cluster O67883 -> trainval



----- decide for cluster A5F5Y3 -----
cluster A5F5Y3 has annotations:
   index     Rep     ChEBI-ID  count
0   1656  A5F5Y3  CHEBI:18420     26
metal CHEBI:18420 exceed maximum, cluster A5F5Y3 -> trainval



----- decide for cluster O70572 -----
cluster O70572 has annotations:
   inde

In [ ]:
print(len(test))

3956


In [ ]:
print(len(trainval))

29367


In [ ]:
list(set(test) & set(trainval)) # check no intersection

[]

## Helper function 
`check_Allmetal` counts, for each metal type, how many residues are bond with that metal in test set, and the proportion: #residues bond by the metal in test set/#residues bond by the metal in full set.

In [ ]:
# for i in cluster_label[cluster_label['Rep'] == 'Q8HUH0']['ChEBI-ID']:
#       print(i)
def check_Allmetal(reps):
  stat = {}
  metals = metal_count_df['ChEBI-ID'].unique()
  metal_dic = dict(zip(metals, [0 for i in range(len(metals))]))
  for rep in reps:
    for i, row in rep_metal[rep_metal['Rep'] == rep].iterrows():
      metal_dic[row['ChEBI-ID']] += row['count']
  for i in metal_dic:
    per = metal_dic[i] / int(metal_count_df[metal_count_df['ChEBI-ID'] == i]['count'])
    stat[i] = [metal_dic[i], per]
    print(f'metal: {i}, num: {metal_dic[i]}, percentage: {per}')
  return stat

def check_metal(seqs, metal):
  cnt = 0
  temp = cluster_label.loc[cluster_label['Accession'].isin(seqs)]
  temp1 = temp['ChEBI-ID'].value_counts().to_frame().reset_index()
  row = temp1[temp1['index'] == metal]['ChEBI-ID']
  cnt = 0 if len(row) == 0 else int(row)
  per = cnt / int(metal_count_df[metal_count_df['ChEBI-ID'] == metal]['count'])
  return per


In [ ]:
init_stat = check_Allmetal(test)

metal: CHEBI:29105, num: 21922, percentage: 0.20009675328824264
metal: CHEBI:18420, num: 16190, percentage: 0.20012855694825582
metal: CHEBI:49883, num: 9982, percentage: 0.2002126080590489
metal: CHEBI:29108, num: 7413, percentage: 0.20029721696838693
metal: CHEBI:29035, num: 4387, percentage: 0.20047525476397204
metal: CHEBI:60240, num: 3551, percentage: 0.2005761409850881
metal: CHEBI:24875, num: 3384, percentage: 0.2006046594344656
metal: CHEBI:190135, num: 1729, percentage: 0.20116346713205352
metal: CHEBI:23378, num: 1411, percentage: 0.20154263676617626
metal: CHEBI:29103, num: 1254, percentage: 0.2016725635252493
metal: CHEBI:49786, num: 677, percentage: 0.19941089837997056
metal: CHEBI:29101, num: 425, percentage: 0.20531400966183574
metal: CHEBI:29034, num: 122, percentage: 0.07389460932768019
metal: CHEBI:30408, num: 54, percentage: 0.05103969754253308
metal: CHEBI:29036, num: 7, percentage: 0.007526881720430108
metal: CHEBI:29033, num: 164, percentage: 0.19759036144578312
m

## replace clusters by proteins in trainval and test set

In [ ]:
test_seqs = []
trainval_seqs = []
test_seqs = list(clusters.loc[clusters['Rep'].isin(test)]['Accession'])
trainval_seqs = list(clusters.loc[clusters['Rep'].isin(trainval)]['Accession'])
# for i in test:
#   for ind, seq in clusters[clusters['Rep'] == i].iterrows():
#     test_seqs.append(seq['Accession'])

# for i in trainval:
#   for ind, seq in clusters[clusters['Rep'] == i].iterrows():
#     trainval_seqs.append(seq['Accession'])
print(test_seqs)
print(trainval_seqs)


['Q4JAP7', 'Q976K1', 'C3MJ44', 'C3MYT6', 'C3MZU1', 'C3N866', 'C3NF46', 'C4KJ24', 'Q980W5', 'Q4L976', 'O07854', 'A9JQL9', 'Q2FDU5', 'Q2FV59', 'Q2YWE7', 'Q5HCY8', 'Q6G6B2', 'Q7A3E1', 'Q8NUQ5', 'Q99R75', 'Q4R736', 'Q7Z572', 'Q68A65', 'Q8BHW6', 'Q4V885', 'A6QP79', 'Q2LK54', 'A5PMY6', 'P02707', 'Q4VRV8', 'A0A0B5L585', 'F1DBB2', 'W6QIT2', 'Q9GZH3', 'Q07152', 'Q5KP44', 'Q6GMG5', 'A0JNA3', 'B0UXP9', 'D3ZLZ7', 'E9PU28', 'F6S675', 'F7CYY5', 'P12268', 'P12269', 'P20839', 'P24547', 'P50096', 'Q3SWY3', 'Q5RGV1', 'O14344', 'P39567', 'P38697', 'P50098', 'P21620', 'Q4WHZ9', 'P50094', 'P50095', 'Q12658', 'Q7SFX7', 'Q54QQ0', 'Q59Q46', 'O00086', 'Q84XA3', 'Q9SA34', 'P47996', 'Q8F4Q4', 'Q4WD45', 'Q50919', 'P22905', 'Q51D88', 'Q52186', 'Q47914', 'Q52720', 'Q8VYP0', 'Q52915', 'Q53239', 'Q60214', 'P38501', 'Q01537', 'Q06006', 'Q92Z29', 'Q54970', 'Q54B14', 'Q54DD2', 'Q54E34', 'Q54E35', 'Q54F10', 'O22769', 'P29914', 'P56909', 'P56910', 'Q9ZDH5', 'Q1RJJ1', 'Q4UM09', 'Q68X20', 'Q92ID9', 'P04394', 'P19234', 'P194

proteins in test set/#proteins in full set, should be roughly 0.1.

In [ ]:
cnt = len(clusters.loc[clusters['Accession'].isin(test_seqs)])

cnt / len(clusters)

0.16103333765582098

In [ ]:
list(set(test_seqs) & set(trainval_seqs)) # check no intersection

[]

In [ ]:
# check if the two sets contain all data
len(test_seqs) + len(trainval_seqs)

177367

## refine the test set
make sure every metal label appears in the test set.

In [ ]:
new_test_seqs = test_seqs.copy()
new_trainval_seqs = trainval_seqs.copy()


for i, v in init_stat.items():
  print(f'check for {i}')
  print(f'initial number is {v[0]}, percentage is {v[1]}')
  if v[1] < 0.199:
    for seq in seqs:
      if seq in new_trainval_seqs:
        if ((seq_metal['Accession'] == seq) & (seq_metal['ChEBI-ID'] == i)).any():
          new_trainval_seqs.remove(seq)
          new_test_seqs.append(seq)
          print(f'{seq} -> test')
          new_per = check_metal(new_test_seqs, i)
          print(f'updated percentage is {new_per}')
          if new_per >= 0.199:
            break

check for CHEBI:29105
initial number is 21922, percentage is 0.20009675328824264
check for CHEBI:18420
initial number is 16190, percentage is 0.20012855694825582
check for CHEBI:49883
initial number is 9982, percentage is 0.2002126080590489
check for CHEBI:29108
initial number is 7413, percentage is 0.20029721696838693
check for CHEBI:29035
initial number is 4387, percentage is 0.20047525476397204
check for CHEBI:60240
initial number is 3551, percentage is 0.2005761409850881
check for CHEBI:24875
initial number is 3384, percentage is 0.2006046594344656
check for CHEBI:190135
initial number is 1729, percentage is 0.20116346713205352
check for CHEBI:23378
initial number is 1411, percentage is 0.20154263676617626
check for CHEBI:29103
initial number is 1254, percentage is 0.2016725635252493
check for CHEBI:49786
initial number is 677, percentage is 0.19941089837997056
check for CHEBI:29101
initial number is 425, percentage is 0.20531400966183574
check for CHEBI:29034
initial number is 122

In [ ]:
list(set(new_test_seqs) & set(new_trainval_seqs)) # check no intersection

[]

## Final proportions

In [ ]:
for metal in metal_count_df['ChEBI-ID'].unique():
  per = check_metal(new_test_seqs, metal) 
  num = int(metal_count_df[metal_count_df['ChEBI-ID'] == metal]['count'])
  print(f'{metal}| num: {int(num*per)} per: {per}')

CHEBI:29105| num: 22031 per: 0.20109166917677557
CHEBI:18420| num: 16194 per: 0.20017800192835422
CHEBI:49883| num: 10033 per: 0.20123553362617086
CHEBI:29108| num: 7413 per: 0.20029721696838693
CHEBI:29035| num: 4427 per: 0.20230315770232601
CHEBI:60240| num: 3551 per: 0.2005761409850881
CHEBI:24875| num: 3384 per: 0.2006046594344656
CHEBI:190135| num: 1729 per: 0.20116346713205352
CHEBI:23378| num: 1411 per: 0.20154263676617626
CHEBI:29103| num: 1254 per: 0.2016725635252493
CHEBI:49786| num: 681 per: 0.20058910162002946
CHEBI:29101| num: 425 per: 0.20531400966183574
CHEBI:29034| num: 334 per: 0.2023016353725015
CHEBI:30408| num: 212 per: 0.2003780718336484
CHEBI:29036| num: 193 per: 0.2075268817204301
CHEBI:29033| num: 168 per: 0.20240963855421687
CHEBI:21137| num: 97 per: 0.22146118721461186
CHEBI:49552| num: 44 per: 0.23157894736842105
CHEBI:48775| num: 80 per: 0.22598870056497175
CHEBI:48828| num: 56 per: 0.25806451612903225
CHEBI:21143| num: 44 per: 0.25
CHEBI:25213| num: 37 per:

In [ ]:
cnt = len(clusters.loc[clusters['Accession'].isin(new_test_seqs)])

cnt / len(clusters)

0.1617324530493271

In [ ]:
len(new_test_seqs)

28686

In [ ]:
len(new_trainval_seqs)

148681

## Write trainval and test set in fasta format
TEST_POS_NEG.fasta: test proteins \
MY_TRAIN_POS_NEG.fasta: trainval proteins

In [ ]:
dataset_metal_binding_summary(new_test_seqs, source = 'POS_TRAIN_FULL.tsv')
pass

total seq in the set: 28686
CHEBI:29105  |Zn(2+)                        |#p:       5282|#residue:  22031|%residue/all: 0.165
CHEBI:18420  |Mg(2+)                        |#p:       7244|#residue:  16194|%residue/all: 0.181
CHEBI:49883  |[4Fe-4S] cluster              |#p:       2688|#residue:  10033|%residue/all: 0.194
CHEBI:29108  |Ca(2+)                        |#p:        972|#residue:   7413|%residue/all: 0.176
CHEBI:29035  |Mn(2+)                        |#p:        794|#residue:   4427|%residue/all: 0.197
CHEBI:60240  |a divalent metal cation       |#p:       1112|#residue:   3551|%residue/all: 0.195
CHEBI:24875  |Fe cation                     |#p:        891|#residue:   3384|%residue/all: 0.194
CHEBI:190135 |[2Fe-2S] cluster              |#p:        620|#residue:   1729|%residue/all: 0.191
CHEBI:23378  |Cu cation                     |#p:        348|#residue:   1411|%residue/all: 0.191
CHEBI:29103  |K(+)                          |#p:        311|#residue:   1254|%residue/all:   0.2
CH

In [ ]:
file_out = 'TEST_POS_NEG.fasta'
write_seq_ls2fasta(file_out, new_test_seqs, 'trimed_combined.fasta')

流式输出内容被截断，只能显示最后 5000 行内容。
writing P0DUM9 to train fasta file.
writing Q56974 to train fasta file.
writing Q5JZR1 to train fasta file.
writing Q6ZFR0 to train fasta file.
writing Q8RXN8 to train fasta file.
writing Q8S2G4 to train fasta file.
writing Q92GB1 to train fasta file.
writing Q9AXE3 to train fasta file.
writing Q9Y6B2 to train fasta file.
writing Q10MC0 to train fasta file.
writing Q12918 to train fasta file.
writing Q1WTS7 to train fasta file.
writing Q21WN2 to train fasta file.
writing Q3JDG3 to train fasta file.
writing Q54EH1 to train fasta file.
writing Q5FA32 to train fasta file.
writing Q6C4C9 to train fasta file.
writing Q7VFP9 to train fasta file.
writing Q863Z5 to train fasta file.
writing Q8D3Z2 to train fasta file.
writing O97578 to train fasta file.
writing P03578 to train fasta file.
writing P06103 to train fasta file.
writing P09466 to train fasta file.
writing P12020 to train fasta file.
writing P27823 to train fasta file.
writing P29415 to train fasta file.
w

In [ ]:
dataset_metal_binding_summary(new_trainval_seqs, source = 'POS_TRAIN_FULL.tsv')
pass

total seq in the set: 148681
CHEBI:29105  |Zn(2+)                        |#p:      19474|#residue:  87526|%residue/all: 0.654
CHEBI:18420  |Mg(2+)                        |#p:      23707|#residue:  64704|%residue/all: 0.724
CHEBI:49883  |[4Fe-4S] cluster              |#p:       8105|#residue:  39824|%residue/all: 0.772
CHEBI:29108  |Ca(2+)                        |#p:       3642|#residue:  29597|%residue/all: 0.702
CHEBI:29035  |Mn(2+)                        |#p:       4195|#residue:  17456|%residue/all: 0.778
CHEBI:60240  |a divalent metal cation       |#p:       3289|#residue:  14153|%residue/all: 0.778
CHEBI:24875  |Fe cation                     |#p:       3869|#residue:  13485|%residue/all: 0.773
CHEBI:190135 |[2Fe-2S] cluster              |#p:       1977|#residue:   6866|%residue/all:  0.76
CHEBI:23378  |Cu cation                     |#p:       1051|#residue:   5590|%residue/all: 0.756
CHEBI:29103  |K(+)                          |#p:       1944|#residue:   4964|%residue/all: 0.792
C

In [ ]:
file_out = 'TRAIN_POS_NEG.fasta'
write_seq_ls2fasta(file_out, new_trainval_seqs, 'trimed_combined.fasta')

流式输出内容被截断，只能显示最后 5000 行内容。
writing A8MHJ4 to train fasta file.
writing A9FGS9 to train fasta file.
writing B1LM67 to train fasta file.
writing B1YJL9 to train fasta file.
writing B2FP04 to train fasta file.
writing B2UNS0 to train fasta file.
writing B6J0W1 to train fasta file.
writing B7K8N3 to train fasta file.
writing B8F6S2 to train fasta file.
writing C5BHX0 to train fasta file.
writing E8MF10 to train fasta file.
writing E9EM69 to train fasta file.
writing F2Z5Z6 to train fasta file.
writing F4JGP4 to train fasta file.
writing F7XKY8 to train fasta file.
writing O07776 to train fasta file.
writing O14576 to train fasta file.
writing O46098 to train fasta file.
writing O46685 to train fasta file.
writing O65333 to train fasta file.
writing O65355 to train fasta file.
writing O67520 to train fasta file.
writing O74020 to train fasta file.
writing O74372 to train fasta file.
writing O95371 to train fasta file.
writing O95379 to train fasta file.
writing P08884 to train fasta file.
w

In [ ]:
!cp TEST_POS_NEG.fasta /content/drive/MyDrive/FYP/TEST_POS_NEG.fasta

In [ ]:
!cp TRAIN_POS_NEG.fasta /content/drive/MyDrive/FYP/TRAIN_POS_NEG.fasta